In [1]:
import pandas as pd
import numpy as np
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import SVD
from surprise import SVDpp
from surprise import Dataset
from surprise.model_selection import GridSearchCV
from sklearn.preprocessing import normalize
import scipy.cluster.hierarchy as shc
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder 
from collections import defaultdict
import torch
from torch import nn
from scipy.sparse import rand as sprand
from sklearn.preprocessing import LabelEncoder

In [3]:
def trans_frame(df,k):
    x = 0
    for col in df.columns[k:]:
        num = len(df[col].value_counts())
        le = LabelEncoder()
        y = le.fit_transform(df[col])
        df[col] = y
        df[col] = df[col] + x
        x += num
    return df

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder 
df = pd.read_csv('frappe.csv', sep="\t")
meta_app = pd.read_csv('meta.csv', sep="\t")

In [5]:
df = pd.read_csv('frappe.csv', sep="\t")
meta_app = pd.read_csv('meta.csv', sep="\t")
df_context = df.copy()
del df_context['cost']
df_context['cnt'] = df['cnt'].astype('float')
# transformation of the number of interactions by log
df_context['cnt'] = df_context['cnt'].apply(np.log10)
a = df_context['user'].value_counts()[df_context['user'].value_counts() >= 10] ## Here we apply the 10-core setting to ensure the quality of the dataset
df_context = df_context[df_context['user'].isin(list(a.index))]
# Here we filter out the items that are excluded by the 10-core setting
meta_app = meta_app[meta_app['item'].isin(
    list(set(df_context['item'].values)))]

In [6]:
le = LabelEncoder()
b = le.fit_transform(meta_app['item'])
i = meta_app['item'].values
dict_co = dict(zip(i, b))
df_context['item'] = df_context['item'].map(dict_co)
meta_app['item'] = b
b = le.fit_transform(df_context['user'])
df_context['user'] = b

# we give index to all the contextual conditions (starting from 0)
df_context = trans_frame(df_context, 3)

In [7]:
meta_app_knowledge = meta_app[['item','category',
                               'downloads', 'language', 'price', 'rating']]
meta_app_knowledge['price'] = np.where(meta_app_knowledge['price'] == 'Free', 'Free',
                                       np.where(meta_app_knowledge['price'] == 'unknown', 'Unknown', 'Paid'))
meta_app_knowledge['rating'][meta_app_knowledge['rating'] == 'unknown'] = 0
meta_app_knowledge['rating'] = meta_app_knowledge['rating'].astype('float')
meta_app_knowledge['rating'] = np.where(meta_app_knowledge['rating'] == 0, 'Unknown',
                                        np.where(meta_app_knowledge['rating'] >= 4.3, 'High_price',
                                                 np.where(meta_app_knowledge['rating'] >= 3.8, 'Mid_price', 'Low_price')))

# we give index to all attributes of items (starting from 0)
meta_app_knowledge = trans_frame(meta_app_knowledge, 1)

<ipython-input-7-3686d9a9f4cb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_app_knowledge['price'] = np.where(meta_app_knowledge['price'] == 'Free', 'Free',
<ipython-input-7-3686d9a9f4cb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_app_knowledge['rating'][meta_app_knowledge['rating'] == 'unknown'] = 0
<ipython-input-7-3686d9a9f4cb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

In [1]:
# hehe_test = df_context.copy()
# df_empty = pd.DataFrame(columns=['user', 'item', 'cnt', 'daytime', 'weekday', 'isweekend', 'homework',
#                                  'weather', 'country', 'city'])
# user_set = set(hehe_test['user'])
# for user in user_set:
#     mean_rating = hehe_test[hehe_test['user'] == user]['cnt'].mean()
#     pos_df = hehe_test[hehe_test['user'] == user]
#     pos_df = pos_df[pos_df['cnt'] > mean_rating]
#     pos_df = pos_df.iloc[-1:]
#     index_to_remove = list(pos_df.iloc[-1:].index)
#     df_empty = df_empty.append(pos_df)
#     hehe_test = hehe_test.drop(index=index_to_remove)

In [11]:
# hehe_test.to_csv('recsys2023/train_no.csv',index=False)
# df_empty.to_csv('recsys2023/test_no.csv',index=False)
# meta_app_knowledge.to_csv('recsys2023/app.csv',index = False)
# df_context.to_csv('recsys2023/all.csv',index = False)

In [12]:
df_usable = df_context.copy()
df_train = df_usable.sample(frac = 0.8, random_state = 0, axis = 0)
df_rest = df_usable[~df_usable.index.isin(df_train.index)]
df_test = df_rest.sample(frac = 0.5, random_state = 0,axis = 0)
df_validation = df_rest[~df_rest.index.isin(df_test.index)]

4.458668059764898